# MHKiT Environmental Contours

Environmental contours describing extreme sea states can be used for numerical or physical model simulations analyzing the design-response of WECs. MHKiT includes functions adapted from the [WDRT](https://github.com/WEC-Sim/WDRT) for creating environmental contours of extreme sea states using a principle components analysis (PCA) methodology with additional improvements for characterizing the joint probability distribution of sea state variables of interest. These environmental contours, characterized by combinations of significant wave height (Hs) and either energy period (Te) or peak period (Tp), provide inputs that are associated with a certain reliability interval. These reliability levels are needed to drive both contour and full sea state style long-term extreme response analyses. This notebook will swalk through the following steps to find a 100 year contour for NDBC buoy 46022.

 1. Request Spectral Wave Density Data from NDBC
 2. Calculate Hs and Te using the NDBC Data
 3. Perform the Principal Component Analysis
 4. Find the contour line for the 100 year 
 5. Plot the results

Start by importing the necessary python packages and MHKiT module.

In [1]:

from mhkit.wave import resource, graphics
import matplotlib.pyplot as plt
from mhkit.wave.io import ndbc
from scipy import stats
import pandas as pd
import numpy as np

## 1. Request Spectral Wave Density Data from NDBC
   
MHKiT can be used to request historical data from the National Data Buoy Center (NDBC). This process is split into 2 steps where first the user requests the available data for a given parameter and optionally a buoy number. We will looking at the function documentation for `mhkit.wave.io.ndbc.available_data`.
`help(ndbc.available_data)`
Looking at the help for the `available_data` function (e.g. `help(ndbc.available_data)`) the function requires a parameter to be specified optionally provide a buoy_number may be provided as a string. For the creating the environmental contour we are interested in historical data `'swden'` and we will specify the buoy number as `'46022'`.

In [2]:
# Specify the parameter to be spectral wave density and the buoy number to be 46022
spectral_wave_density='swden'
buoy_number = '46022' 
available_data= ndbc.available_data(spectral_wave_density, buoy_number)
available_data.tail()

,id,year,filename
1434,46022,2016,46022w2016.txt.gz
1435,46022,2017,46022w2017.txt.gz
1436,46022,2018,46022w2018.txt.gz
1437,46022,2019,46022w2019.txt.gz
1438,46022,2004,46022wb2004.txt.gz


The `ndbc.available_data` function has returned a dataframe with columns 'id', 'year', and 'filename'. All three of these columns are of type string because buoy_numbers (id) are defined as 5 digit alpha number specifiers, years sometimes split (e.g. 'b2004').  In this case the years span from 1996 to 2019 and for demonstration we have decided we are interested in the data between years 1996 and 2012. Because the list needs to contain `'b2004'` 

In [3]:
# Create a list of years of interest
subYears = [*range(1996,2013)]
# Slice the DataFrame on years of interest
yearsOfInterest = available_data[available_data.year.isin(subYears)]
yearsOfInterest.tail()

,id,year,filename
1427,46022,2009,46022w2009.txt.gz
1428,46022,2010,46022w2010.txt.gz
1429,46022,2011,46022w2011.txt.gz
1430,46022,2012,46022w2012.txt.gz
1438,46022,2004,46022wb2004.txt.gz


Using the filenames from our `yearsOfInterest` we can request data from NDC and return a dictionary of DataFrames indexed by years.

In [4]:
# Get dictionary of parameter data by year
filenames= yearsOfInterest['filename']
ndbc_data = ndbc.request_data(spectral_wave_density, filenames)

# Lastly we will convert a DateTime Index 
mhkit_data={}
# Create a Datetime Index and remove NOAA date columns for each year
for year in ndbc_data:
   year_data = ndbc_data[year]
   year_data['date'], ndbc_date_cols = ndbc.dates_to_datetime(spectral_wave_density, 
                                                              year_data, 
                                                              return_date_cols=True)
   year_data = year_data.drop(ndbc_date_cols, axis=1)
   year_data = year_data.set_index('date')
   # Convert columns to float now that the ndbc_date_cols (type=str) are gone
   year_data.columns = year_data.columns.astype(float)     
   mhkit_data[year] = year_data

## 2. Calculate Hs and Te using the NDBC Data



In [5]:
#=======================================================================
# Calculate Hm0 and Te for each year
#=======================================================================
Hs={}
Te={}
for year in mhkit_data:
    year_data = mhkit_data[year]
    Hs[year] = resource.significant_wave_height(year_data.T)
    Te[year] = resource.energy_period(year_data.T)


Hs_list = [ v for k,v in Hs.items()]; 
Te_list = [ v for k,v in Te.items()]; 

Hs= pd.concat(Hs_list ,axis=0)
Te= pd.concat(Te_list ,axis=0)

Hs['Te'] = Te.Te
Hs.dropna(inplace=True)

# Save the data locally
Hs.to_pickle(f'{buoy_number}Hm0Te.pkl')

## 3. Perform the Principal Component Analysis 


In [10]:
df_raw=Hs.copy(deep=True)

# Remove Outliers
df = df_raw[df_raw['Hm0'] < 20]

Hm0 = df.Hm0.values  
Te = df.Te.values 

# Delta time of sea-states 
dt_ss = (df.index[2]-df.index[1]).seconds  
# Return periods (yrs) of interest
contour_year = 100  
Hs_Return, T_Return = resource.environmental_contour(Hm0, Te, time_SS, time_R)

1.0

## 4. Find the contour line for the 100 year

In [7]:
# Declare required parameters
time_SS = 1.  # Sea state duration (hrs)
time_R = 100  # Return periods (yrs) of interest
Hm0_contour, T_contour = resource.environmental_contour(time_SS, time_R, PCA , nb_steps=1000)

NameError: name 'PCA' is not defined

## 5. Plot overlay of the data and Contour


In [ ]:
graphics.plot_environmental_contour(df.Hm0.values, df.Te.values, 
                                    Hm0_contour, T_contour, 
                                    data_label='NDBC 46022', 
                                    contour_label='100 Year Contour',
                                    ax=None)